In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import lightgbm as lgb
import torch
import random
import re

import utils

In [3]:
class CFG:
    make_folder = True
    note_num = "StackingOpt/EDA007"
    seed = 42


OUTPUT_DIR = f'H:/study/output/{CFG.note_num}/'

if CFG.make_folder:
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)


# error_dataのオプティカルフローの予測精度

In [3]:
def get_file_paths(folder_path):
    file_paths = []
    for root, directories, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            file_paths.append(file_path)
    return file_paths


folder_path = r"H:\study\error_data\kanto_parameter_lambda_0.019_iterate_1000_p3\7月"
file_paths = get_file_paths(folder_path)


In [75]:
df_list = []
for path in file_paths:
    match = re.search(r'(\d+)月(\d+)日', path.split("\\")[-2])
    month = match.group(1)
    day = match.group(2)
    year = "20" + path.split("\\")[-1].split("_")[1][:2]
    hour = path.split("\\")[-1].split("_")[1][2+len(month)+len(day):].split(".")[0]
    minute = int(path.split("\\")[-1].split("_")[1][2+len(month)+len(day):].split(".")[1])*60/10
    date = datetime(int(year), int(month), int(day), int(hour), int(minute))

    df = pd.read_csv(path,encoding='cp932')
    df["datetime"] = date
    df_list.append(df)
df = pd.concat(df_list)
df["nv*twoweeks_max"] = df["nv"] * df["twoweeks_max"]
df["env*twoweeks_max"] = df["env"] * df["twoweeks_max"]

In [80]:
missing_rate = df.groupby('id')['nv'].apply(lambda x: x.isna().mean()).reset_index(name='missing_rate')
missing_rate.sort_values(by="missing_rate")

,id,missing_rate
51,1680000228,0.0
58,2220000003,0.0
57,2220000002,0.0
56,2220000001,0.0
54,1680000327,0.0
...,...,...
39,1680000081,1.0
101,6910000469,1.0
52,1680000285,1.0
19,1160000420,1.0


In [81]:
id = 1680000228
((df.loc[df.id==id,"nv*twoweeks_max"]-df.loc[df.id==id,"env*twoweeks_max"]).abs()/df.loc[df.id==id,"observed_max"]).mean()*100

6.774886971885137

In [82]:
df2 = df[df.id==id]

# EDA006で保存したオプティカルフローの予測精度
- error_dataの予測精度とほぼ一致する。→メッシュデータから正しく変換できている。

In [23]:
start_date = "201407010000"
end_date = "201408010000"
data_list = utils.create_time_series_data(start_date,end_date)

save_file_list = []
for date in data_list:
    save_file = pd.read_csv('H:/study/output/StackingOpt/EDA006/'+f"pred_id_all_data_{date}.csv")
    save_file_list.append(save_file)
save_files = pd.concat(save_file_list)

In [89]:
save_files2 = save_files[save_files.id==id]

In [95]:
save_files2[save_files2.datetime=="2014-07-01 08:00:00"]

,id,pred,pred_interpolated,datetime
1737,1680000228,0.934254,0.93425,2014-07-01 08:00:00


In [94]:
df2[df2.datetime=="2014-07-01 08:00:00"]

,id,id_lat,id_lng,id_lat_mesh,id_lng_mesh,id_prefecture,pvrate,observed_max,twoweeks_max,flo_lng,...,nv_meshint,env,preal,preal_meshint,ppred,nv_max_perror,nv_meshint_max_perror,datetime,nv*twoweeks_max,env*twoweeks_max
51,1680000228,35.7707,139.9887,35.78,139.98,千葉県,10.2,8570.962,6431.8,-0.074745,...,0.541,0.936,3415.286,3481.726,6019.836,30.388,29.613,2014-07-01 08:00:00,3415.2858,6020.1648


In [96]:
df2["datetime"] = pd.to_datetime(df2["datetime"])
save_files2["datetime"] = pd.to_datetime(save_files2["datetime"])

df3 = df2.merge(save_files2,on=["id","datetime"],how="left")

df3["pred*twoweeks_max"] = df3["pred"] * df3["twoweeks_max"]
df3["pred_interpolated*twoweeks_max"] = df3["pred_interpolated"] * df3["twoweeks_max"]

C:\Users\Yosui\AppData\Local\Temp\ipykernel_19212\3560382504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["datetime"] = pd.to_datetime(df2["datetime"])
C:\Users\Yosui\AppData\Local\Temp\ipykernel_19212\3560382504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  save_files2["datetime"] = pd.to_datetime(save_files2["datetime"])


In [98]:
((df3["nv*twoweeks_max"]-df3["env*twoweeks_max"]).abs()/df3["observed_max"]).mean()*100

6.774886971885137

In [100]:
((df3["nv*twoweeks_max"]-df3["pred*twoweeks_max"]).abs()/df3["observed_max"]).mean()*100

6.761735866179801

In [101]:
((df3["nv*twoweeks_max"]-df3["pred_interpolated*twoweeks_max"]).abs()/df3["observed_max"]).mean()*100

6.761727091883738

In [102]:
save_file 

,id,pred,pred_interpolated,datetime
0,6,0.532696,0.53270,2014-07-31 18:00:00
1,9,0.232816,0.23282,2014-07-31 18:00:00
2,12,0.665454,0.66545,2014-07-31 18:00:00
3,14,0.232200,0.23220,2014-07-31 18:00:00
4,17,0.409859,0.40986,2014-07-31 18:00:00
...,...,...,...,...
5092,6990000048,0.768087,0.76809,2014-07-31 18:00:00
5093,6990000049,0.768087,0.76809,2014-07-31 18:00:00
5094,6990000050,0.525790,0.52579,2014-07-31 18:00:00
5095,6990000051,0.525790,0.52579,2014-07-31 18:00:00


# def get_preprocessing_data(to_unique_id)より検証
- こちらでも整合性がとれている。

In [109]:
#unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = ["1680000228"]
unique_id = [int(i) for i in unique_id]
to_unique_id = [str(num).zfill(10) for num in unique_id]
df = utils.get_preprocessing_data(to_unique_id)

df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=["id","datetime"],inplace=True)

In [110]:
df[["id","datetime",'two_weeks_max',"nv","generation"]]

,id,datetime,two_weeks_max,nv,generation
0,1680000228,2013-08-01 00:00:00,0.0,0.0,0.0
1,1680000228,2013-08-01 00:30:00,0.0,0.0,0.0
2,1680000228,2013-08-01 01:00:00,0.0,0.0,0.0
3,1680000228,2013-08-01 01:30:00,0.0,0.0,0.0
4,1680000228,2013-08-01 02:00:00,0.0,0.0,0.0
...,...,...,...,...,...
17611,1680000228,2014-07-31 21:30:00,0.0,0.0,0.0
17612,1680000228,2014-07-31 22:00:00,0.0,0.0,0.0
17613,1680000228,2014-07-31 22:30:00,0.0,0.0,0.0
17614,1680000228,2014-07-31 23:00:00,0.0,0.0,0.0


In [112]:
df3 = save_files2.merge(df[["id","datetime",'two_weeks_max',"nv","generation"]],on=["id","datetime"],how="left")

In [114]:
df3.isnull().sum()

id                   0
pred                 0
pred_interpolated    0
datetime             0
two_weeks_max        0
nv                   0
generation           0
dtype: int64

In [131]:
id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")
df3["observed_max2"] = id_all_data.loc[id_all_data.id==id,"observed_max2"].item()
df3["pred*two_weeks_max"] = df3["pred"] * df3["two_weeks_max"]
df3["pred_interpolated*two_weeks_max"] = df3["pred_interpolated"] * df3["two_weeks_max"]

In [133]:
((df3["generation"]-df3["pred_interpolated*two_weeks_max"]).abs()/df3["observed_max2"]).mean()*100

6.658988183530888

In [134]:
((df3["generation"]-df3["pred*two_weeks_max"]).abs()/df3["observed_max2"]).mean()*100

6.658996824508703

# ユニークidにおける分析
- ユニークidでもおおよそ一致

In [144]:
df_list = []
for path in file_paths:
    match = re.search(r'(\d+)月(\d+)日', path.split("\\")[-2])
    month = match.group(1)
    day = match.group(2)
    year = "20" + path.split("\\")[-1].split("_")[1][:2]
    hour = path.split("\\")[-1].split("_")[1][2+len(month)+len(day):].split(".")[0]
    minute = int(path.split("\\")[-1].split("_")[1][2+len(month)+len(day):].split(".")[1])*60/10
    date = datetime(int(year), int(month), int(day), int(hour), int(minute))

    df = pd.read_csv(path,encoding='cp932')
    df["datetime"] = date
    df_list.append(df)
df = pd.concat(df_list)
df["nv*twoweeks_max"] = df["nv"] * df["twoweeks_max"]
df["env*twoweeks_max"] = df["env"] * df["twoweeks_max"]
df["APE"] = (df["nv*twoweeks_max"]-df["env*twoweeks_max"]).abs()/df["observed_max"]
print("MAPE",df["APE"].mean()*100)

MAPE 6.704960652131726


In [151]:
unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
print("MAPE",df.loc[df.id.isin(unique_id),"APE"].mean()*100)


MAPE 6.704960652131726


In [152]:
start_date = "201407010000"
end_date = "201408010000"
data_list = utils.create_time_series_data(start_date,end_date)

save_file_list = []
for date in data_list:
    save_file = pd.read_csv('H:/study/output/StackingOpt/EDA006/'+f"pred_id_all_data_{date}.csv")
    save_file_list.append(save_file)
save_files = pd.concat(save_file_list)

In [155]:
unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
to_unique_id = [str(num).zfill(10) for num in unique_id]
df = utils.get_preprocessing_data(to_unique_id)

df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=["id","datetime"],inplace=True)

save_files2 = save_files[save_files.id.isin(unique_id)]
save_files2["datetime"] = pd.to_datetime(save_files2["datetime"])
df["datetime"] = pd.to_datetime(df["datetime"])
df3 = save_files2.merge(df[["id","datetime",'two_weeks_max',"nv","generation"]],on=["id","datetime"],how="left")

id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")
df3 = df3.merge(id_all_data[["id","observed_max2"]],on=["id"],how="left")
df3["observed_max2"] = id_all_data.loc[id_all_data.id==id,"observed_max2"].item()
df3["pred*two_weeks_max"] = df3["pred"] * df3["two_weeks_max"]
df3["pred_interpolated*two_weeks_max"] = df3["pred_interpolated"] * df3["two_weeks_max"]

In [167]:
((df3["generation"]-df3["pred_interpolated*two_weeks_max"]).abs()/df3["observed_max2"]).mean()*100

6.708035546920488

In [168]:
((df3["generation"]-df3["pred*two_weeks_max"]).abs()/df3["observed_max2"]).mean()*100

6.708040441334433

# 全てのデータにおける予測値の欠損率の割合を調べる。

In [5]:
start_date = "201308150000"
end_date = "201408010000"
data_list = utils.create_time_series_data(start_date,end_date)

save_file_list = []
for date in data_list:
    save_file = pd.read_csv('H:/study/output/StackingOpt/EDA006/'+f"pred_id_all_data_{date}.csv")
    save_file_list.append(save_file)
save_files = pd.concat(save_file_list)

In [188]:
save_files.isnull().sum()

id                         0
pred                 2529492
pred_interpolated    2205782
datetime                   0
dtype: int64

In [214]:
missing_rates = (save_files.groupby('id')['pred_interpolated'].apply(lambda x: x.isna().mean()) * 100).reset_index(name='missing_rate')
missing_rates[missing_rates.missing_rate==0]


,id,missing_rate
3,14,0.0
5,18,0.0
8,24,0.0
9,28,0.0
10,29,0.0
...,...,...
5044,6970000005,0.0
5045,6970000006,0.0
5046,6980000004,0.0
5047,6980000005,0.0


In [219]:
missing_rates[missing_rates["missing_rate"]<0.2].sort_values(ascending=False,by="missing_rate")

,id,missing_rate
866,1160000124,0.198192
1639,1680000127,0.198192
1174,1230000005,0.198192
1173,1230000004,0.198192
4137,6890000091,0.198192
...,...,...
1918,1690000062,0.000000
1919,1690000063,0.000000
1920,1690000064,0.000000
1921,1690000065,0.000000


In [195]:
missing_rates = (save_files.groupby('id')['pred'].apply(lambda x: x.isna().mean()) * 100).reset_index(name='missing_rate')
missing_rates[missing_rates.missing_rate==0]


,id,missing_rate
5,18,0.0
47,520,0.0
58,544,0.0
60,547,0.0
81,10000021,0.0
...,...,...
4866,6910000380,0.0
4933,6910000448,0.0
4934,6910000449,0.0
4942,6910000457,0.0


In [1]:
# 'pred_interpolated'の欠損が0のid
unique_id2 = [14, 18, 24, 28, 29, 37, 46, 47, 48, 128, 145, 148, 159, 165, 209, 227, 231, 282, 283, 361, 413, 460, 478, 514, 520, 542, 544, 547, 564, 568, 10000002, 10000005, 10000007, 10000008, 10000010, 10000011, 10000014, 10000016, 10000020, 10000021, 10000031, 10000055, 10000059, 10000066, 10000076, 10000093, 10000094, 10000096, 10000099, 10000100, 10000101, 10000102, 10000106, 10000268, 10000287, 1010000001, 1010000002, 1010000003, 1010000004, 1010000005, 1010000006, 1010000007, 1010000008, 1010000009, 1010000010, 1010000011, 1010000012, 1010000013, 1010000014, 1010000015, 1010000016, 1010000017, 1010000018, 1010000019, 1010000020, 1010000021, 1010000022, 1010000023, 1010000024, 1010000025, 1010000026, 1010000027, 1010000028, 1010000029, 1010000030, 1010000031, 1010000032, 1010000033, 1010000034, 1010000035, 1010000036, 1010000037, 1010000038, 1010000039, 1010000040, 1010000041, 1010000042, 1010000043, 1010000044, 1010000046, 1010000047, 1010000048, 1010000049, 1010000050, 1010000051, 1010000052, 1010000053, 1010000054, 1010000055, 1010000056, 1010000057, 1010000058, 1010000059, 1010000060, 1010000061, 1010000062, 1010000063, 1010000064, 1010000065, 1010000066, 1010000067, 1010000068, 1010000069, 1010000070, 1010000071, 1010000072, 1010000073, 1010000074, 1010000075, 1010000076, 1010000077, 1010000078, 1010000079, 1010000080, 1010000081, 1010000082, 1010000083, 1010000084, 1010000085, 1010000086, 1010000087, 1010000088, 1010000089, 1010000090, 1010000091, 1010000092, 1010000093, 1010000094, 1010000095, 1010000096, 1010000097, 1010000098, 1010000099, 1010000100, 1010000101, 1010000102, 1010000103, 1010000104, 1010000105, 1010000106, 1010000107, 1010000108, 1010000109, 1010000110, 1010000111, 1010000112, 1010000113, 1010000114, 1010000115, 1010000116, 1010000117, 1010000118, 1010000119, 1010000120, 1010000121, 1010000122, 1010000123, 1010000124, 1010000125, 1010000126, 1010000127, 1010000128, 1010000130, 1010000131, 1010000132, 1010000133, 1010000134, 1010000135, 1010000136, 1010000137, 1010000138, 1010000139, 1010000140, 1010000141, 1010000142, 1010000143, 1010000144, 1010000145, 1010000147, 1010000148, 1010000149, 1010000150, 1010000151, 1010000152, 1010000153, 1010000154, 1010000155, 1010000156, 1010000157, 1010000158, 1010000159, 1010000160, 1010000161, 1010000162, 1010000163, 1010000164, 1010000165, 1010000166, 1010000167, 1010000168, 1010000169, 1010000170, 1010000171, 1010000172, 1010000173, 1010000174, 1010000175, 1010000176, 1010000177, 1010000178, 1010000179, 1010000180, 1010000181, 1010000182, 1010000183, 1010000184, 1010000185, 1010000186, 1010000187, 1010000188, 1010000189, 1010000190, 1010000191, 1010000192, 1010000193, 1010000194, 1010000195, 1010000196, 1010000197, 1010000198, 1010000199, 1010000200, 1010000201, 1010000202, 1010000203, 1010000204, 1010000205, 1010000206, 1010000207, 1010000208, 1010000209, 1010000210, 1010000211, 1010000212, 1010000213, 1010000214, 1010000215, 1010000216, 1010000217, 1010000218, 1010000219, 1010000220, 1010000221, 1010000222, 1010000223, 1010000224, 1010000225, 1010000226, 1010000227, 1010000228, 1010000229, 1010000230, 1010000231, 1010000232, 1010000233, 1010000234, 1010000235, 1010000236, 1010000237, 1010000238, 1010000239, 1010000240, 1010000241, 1010000242, 1010000243, 1010000244, 1010000245, 1010000246, 1010000247, 1010000248, 1010000249, 1010000250, 1010000251, 1010000252, 1010000253, 1010000257, 1010000258, 1010000259, 1010000260, 1010000262, 1010000263, 1010000264, 1010000265, 1010000266, 1010000268, 1010000271, 1010000272, 1010000273, 1010000274, 1010000275, 1010000276, 1010000277, 1010000278, 1010000279, 1010000280, 1010000281, 1010000292, 1010000295, 1010000296, 1010000297, 1010000298, 1010000299, 1010000300, 1010000301, 1010000302, 1010000303, 1010000304, 1010000305, 1010000306, 1010000307, 1010000308, 1010000309, 1010000310, 1010000311, 1010000312, 1010000313, 1010000314, 1010000315, 1010000316, 1010000317, 1010000318, 1010000319, 1010000321, 1010000322, 1010000323, 1010000324, 1010000325, 1010000326, 1010000327, 1010000328, 1010000329, 1010000330, 1010000331, 1010000332, 1010000333, 1010000334, 1010000335, 1010000336, 1010000337, 1010000338, 1010000339, 1010000340, 1010000341, 1010000342, 1010000343, 1010000344, 1010000345, 1010000346, 1010000347, 1010000348, 1010000351, 1010000352, 1010000353, 1010000354, 1010000355, 1010000356, 1010000357, 1010000358, 1010000362, 1020000001, 1030000001, 1040000002, 1040000003, 1040000004, 1040000006, 1040000008, 1040000010, 1040000011, 1040000014, 1040000015, 1040000018, 1040000020, 1040000024, 1040000027, 1040000034, 1040000035, 1040000039, 1040000040, 1040000041, 1040000045, 1040000046, 1040000047, 1040000049, 1040000053, 1040000058, 1040000065, 1080000001, 1080000002, 1080000004, 1080000005, 1080000006, 1160000002, 1160000003, 1160000005, 1160000006, 1160000009, 1160000012, 1160000016, 1160000020, 1160000021, 1160000022, 1160000023, 1160000024, 1160000026, 1160000027, 1160000028, 1160000029, 1160000030, 1160000031, 1160000033, 1160000034, 1160000035, 1160000036, 1160000037, 1160000038, 1160000040, 1160000041, 1160000042, 1160000043, 1160000044, 1160000045, 1160000046, 1160000047, 1160000048, 1160000049, 1160000052, 1160000053, 1160000054, 1160000055, 1160000057, 1160000059, 1160000060, 1160000082, 1160000085, 1160000086, 1160000088, 1160000092, 1160000093, 1160000094, 1160000095, 1160000096, 1160000097, 1160000098, 1160000099, 1160000101, 1160000105, 1160000106, 1160000107, 1160000108, 1160000109, 1160000112, 1160000113, 1160000114, 1160000115, 1160000116, 1160000118, 1160000119, 1160000120, 1160000121, 1160000122, 1160000125, 1160000127, 1160000129, 1160000130, 1160000131, 1160000132, 1160000134, 1160000135, 1160000136, 1160000137, 1160000138, 1160000139, 1160000140, 1160000142, 1160000156, 1160000157, 1160000159, 1160000160, 1160000166, 1160000167, 1160000188, 1160000189, 1160000190, 1160000191, 1160000192, 1160000193, 1160000194, 1160000195, 1160000196, 1160000197, 1160000198, 1160000199, 1160000200, 1160000204, 1160000205, 1160000207, 1160000208, 1160000211, 1160000213, 1160000218, 1160000220, 1160000221, 1160000222, 1160000223, 1160000225, 1160000226, 1160000227, 1160000228, 1160000229, 1160000230, 1160000231, 1160000232, 1160000233, 1160000234, 1160000235, 1160000236, 1160000237, 1160000238, 1160000239, 1160000240, 1160000241, 1160000242, 1160000245, 1160000246, 1160000247, 1160000248, 1160000249, 1160000250, 1160000251, 1160000252, 1160000254, 1160000258, 1160000259, 1160000260, 1160000261, 1160000262, 1160000264, 1160000265, 1160000266, 1160000267, 1160000268, 1160000269, 1160000270, 1160000271, 1160000272, 1160000273, 1160000274, 1160000277, 1160000278, 1160000279, 1160000280, 1160000281, 1160000282, 1160000283, 1160000284, 1160000285, 1160000286, 1160000291, 1160000297, 1160000298, 1160000299, 1160000300, 1160000319, 1160000321, 1160000323, 1160000342, 1160000345, 1160000346, 1160000348, 1160000362, 1160000367, 1160000412, 1160000431, 1160000433, 1160000447, 1160000448, 1160000591, 1160000592, 1180000002, 1180000045, 1200000008, 1200000009, 1210000001, 1210000002, 1210000003, 1210000004, 1210000005, 1210000006, 1210000007, 1210000008, 1210000009, 1210000010, 1210000011, 1210000013, 1210000014, 1210000018, 1210000055, 1210000056, 1210000057, 1210000058, 1210000059, 1210000060, 1210000065, 1210000066, 1210000067, 1220000002, 1220000003, 1220000004, 1220000006, 1230000001, 1230000003, 1230000018, 1230000019, 1240000002, 1240000003, 1240000004, 1240000005, 1260000001, 1270000002, 1270000003, 1270000005, 1270000008, 1270000009, 1270000010, 1270000011, 1270000013, 1270000014, 1270000015, 1270000016, 1270000019, 1270000020, 1270000022, 1270000037, 1270000038, 1290000002, 1290000003, 1290000004, 1290000005, 1290000006, 1290000007, 1290000008, 1290000009, 1290000012, 1290000013, 1300000001, 1320000001, 1320000002, 1320000003, 1330000001, 1330000002, 1330000010, 1400000001, 1400000002, 1400000003, 1400000004, 1400000005, 1410000001, 1410000002, 1410000003, 1410000004, 1420000001, 1420000002, 1420000003, 1430000002, 1450000001, 1450000002, 1450000003, 1450000004, 1460000001, 1480000001, 1480000003, 1510000001, 1510000002, 1510000003, 1570000001, 1580000001, 1580000003, 1580000004, 1580000005, 1610000002, 1610000003, 1610000004, 1610000005, 1610000006, 1610000007, 1610000008, 1650000001, 1660000003, 1660000004, 1660000005, 1660000006, 1660000007, 1660000008, 1660000009, 1660000010, 1660000011, 1660000012, 1660000013, 1660000014, 1660000015, 1660000016, 1660000017, 1670000001, 1670000002, 1670000003, 1670000004, 1670000007, 1680000055, 1680000056, 1680000094, 1680000125, 1680000126, 1680000168, 1680000192, 1680000209, 1680000210, 1680000211, 1680000212, 1680000254, 1680000265, 1680000267, 1680000312, 1680000313, 1680000362, 1690000001, 1690000002, 1690000003, 1690000004, 1690000005, 1690000006, 1690000007, 1690000008, 1690000009, 1690000010, 1690000011, 1690000012, 1690000013, 1690000014, 1690000015, 1690000017, 1690000018, 1690000019, 1690000020, 1690000021, 1690000022, 1690000023, 1690000024, 1690000025, 1690000026, 1690000027, 1690000028, 1690000029, 1690000030, 1690000031, 1690000032, 1690000033, 1690000034, 1690000035, 1690000036, 1690000037, 1690000038, 1690000039, 1690000040, 1690000041, 1690000042, 1690000043, 1690000044, 1690000045, 1690000046, 1690000047, 1690000048, 1690000049, 1690000050, 1690000051, 1690000052, 1690000053, 1690000054, 1690000056, 1690000057, 1690000058, 1690000059, 1690000060, 1690000061, 1690000062, 1690000063, 1690000064, 1690000065, 1690000066, 1690000067, 1690000068, 1690000069, 1690000070, 1690000071, 1690000072, 1690000073, 1690000074, 1690000075, 1690000076, 1690000078, 1690000079, 1690000080, 1690000081, 1690000082, 1690000083, 1690000084, 1690000085, 1690000086, 1690000087, 1690000088, 1690000089, 1690000090, 1690000091, 1690000092, 1690000093, 1690000094, 1690000095, 1690000096, 1690000097, 1690000098, 1690000099, 1690000100, 1690000101, 1690000102, 1690000103, 1690000104, 1690000105, 1690000106, 1690000110, 1690000111, 1690000112, 1690000113, 1690000114, 1690000115, 1690000116, 1690000117, 1690000118, 1690000119, 1690000120, 1690000121, 1690000122, 1690000123, 1690000124, 1690000125, 1690000126, 1690000127, 1690000128, 1690000129, 1690000132, 1690000133, 1690000134, 1690000135, 1690000136, 1690000137, 1690000138, 1690000139, 1690000140, 1690000141, 1690000142, 1690000143, 1690000145, 1690000147, 1690000150, 1690000151, 1690000152, 1690000153, 1690000155, 1690000156, 1690000158, 1690000161, 1690000162, 1690000164, 1690000167, 1690000168, 1690000172, 1690000174, 1690000176, 1690000177, 1690000178, 1690000179, 1690000180, 1690000181, 1690000182, 1690000184, 1690000186, 1690000190, 1690000198, 1690000199, 1690000203, 1690000206, 1690000209, 1710000002, 1710000003, 1710000004, 1710000005, 1710000006, 1710000007, 1710000008, 1710000009, 1710000010, 1710000011, 1730000001, 1730000002, 1730000003, 1730000004, 1730000005, 1730000006, 1730000007, 1730000008, 1730000009, 1730000010, 1730000011, 1730000012, 1730000013, 1730000014, 1730000018, 1730000019, 1730000020, 1730000021, 1740000001, 1750000001, 1750000002, 1750000003, 1750000004, 1750000005, 1750000006, 1750000007, 1750000008, 1750000009, 1750000010, 1750000011, 1750000012, 1750000013, 1760000001, 1760000002, 1760000003, 1770000001, 1770000015, 1770000016, 1770000017, 1770000018, 1770000019, 1770000027, 1780000003, 1780000004, 1780000005, 1790000001, 1810000001, 1810000005, 1840000001, 1840000002, 1840000003, 1850000001, 1850000002, 1850000003, 1890000001, 1900000001, 1900000002, 1900000003, 1930000001, 2010000001, 2010000006, 2010000010, 2010000132, 2010000155, 2010000156, 2010000157, 2010000185, 2020000002, 2030000003, 2030000004, 2030000005, 2030000006, 2030000007, 2030000008, 2040000021, 2040000024, 2040000031, 2040000039, 2040000044, 2040000138, 2040000141, 2040000147, 2040000153, 2040000191, 2040000203, 2050000001, 2070000001, 2070000002, 2070000003, 2070000004, 2070000005, 2070000006, 2070000007, 2070000008, 2100000001, 2100000035, 2100000066, 2100000067, 2100000068, 2100000069, 2100000070, 2100000071, 2100000084, 2100000085, 2100000086, 2100000184, 2100000185, 2110000001, 2110000002, 2110000003, 2110000004, 2110000005, 2130000001, 2130000002, 2130000003, 2130000004, 2150000001, 2150000002, 2190000001, 2190000002, 2190000003, 2210000001, 2210000002, 2210000003, 2210000004, 2210000005, 2210000006, 2230000001, 2230000002, 2230000003, 2340000001, 2350000001, 2350000002, 2360000001, 2360000002, 2360000005, 2360000006, 2390000001, 2390000002, 2390000003, 2390000004, 2390000005, 2410000001, 2410000002, 2410000003, 2410000004, 2410000005, 2410000006, 2410000007, 2410000009, 2410000010, 2410000011, 2410000012, 2410000015, 2410000016, 2410000017, 2420000001, 2420000006, 2430000001, 2430000002, 2430000003, 2430000004, 2430000005, 2430000006, 2440000003, 2440000004, 2440000005, 2440000006, 2440000007, 2440000008, 2440000009, 2440000010, 2440000011, 2440000012, 2440000013, 2440000014, 2450000001, 2450000002, 2450000003, 2460000001, 2460000002, 2460000003, 2460000004, 2460000005, 2460000006, 2460000007, 2460000008, 2460000009, 2460000010, 2460000011, 2460000012, 2460000013, 2460000014, 2470000002, 2470000003, 2470000005, 2500000006, 2500000007, 2500000008, 2520000003, 2520000004, 2520000005, 2540000001, 2550000001, 2550000002, 2720000005, 2720000008, 2720000011, 2720000012, 2720000013, 2720000017, 2760000004, 2760000005, 2760000006, 2900000001, 2900000002, 3000000004, 3000000025, 3000000122, 3000000127, 3000000130, 3000000132, 3000000137, 3000000138, 3000000139, 3000000141, 3000000142, 3000000143, 3000000147, 3000000152, 3000000153, 3000000158, 3000000159, 3000000162, 3000000163, 3000000164, 3000000172, 3000000173, 3000000174, 3000000175, 3000000176, 3000000177, 3000000178, 3000000179, 3000000180, 3000000184, 3000000194, 3000000195, 3000000196, 3000000197, 3000000198, 3000000201, 3000000202, 3000000203, 3000000204, 3000000205, 3000000206, 3000000207, 3000000208, 3000000209, 3000000211, 3000000222, 5000000001, 5000000022, 5000000023, 5000000024, 5000000028, 5000000029, 5000000030, 5000000063, 5000000064, 5000000065, 5000000074, 5000000075, 5000000079, 5000000095, 5000000096, 5000000102, 5000000105, 5000000107, 5000000115, 5000000116, 5000000118, 5000000119, 5000000121, 5000000128, 5000000129, 5000000130, 5000000145, 6010000001, 6010000002, 6010000003, 6020000001, 6020000002, 6020000003, 6030000001, 6030000002, 6030000003, 6030000004, 6030000005, 6030000006, 6040000068, 6040000070, 6040000072, 6040000073, 6040000074, 6040000075, 6040000076, 6040000077, 6040000078, 6040000079, 6040000080, 6040000081, 6040000082, 6040000085, 6040000086, 6040000095, 6040000097, 6050000001, 6050000002, 6060000015, 6060000019, 6060000026, 6060000027, 6070000001, 6070000002, 6070000003, 6070000004, 6070000005, 6070000006, 6070000007, 6070000008, 6070000009, 6070000010, 6070000011, 6070000012, 6070000013, 6070000014, 6070000015, 6070000020, 6070000033, 6070000034, 6070000035, 6070000036, 6070000037, 6070000038, 6070000039, 6070000040, 6070000041, 6070000042, 6070000043, 6070000050, 6070000051, 6070000052, 6070000053, 6070000054, 6070000055, 6070000056, 6070000073, 6070000074, 6070000075, 6070000096, 6070000097, 6070000098, 6080000002, 6080000003, 6080000004, 6080000005, 6080000006, 6080000008, 6090000013, 6090000035, 6090000036, 6090000037, 6090000040, 6090000045, 6090000046, 6090000047, 6090000054, 6090000060, 6090000061, 6090000062, 6090000063, 6090000064, 6090000065, 6090000066, 6090000067, 6090000076, 6090000085, 6090000086, 6090000087, 6090000088, 6090000089, 6090000090, 6090000091, 6090000092, 6090000097, 6090000098, 6100000001, 6110000003, 6110000004, 6110000005, 6110000007, 6110000008, 6110000010, 6110000011, 6110000017, 6110000020, 6110000021, 6110000023, 6110000025, 6110000026, 6110000027, 6110000029, 6110000034, 6110000035, 6110000039, 6110000040, 6110000041, 6110000042, 6110000043, 6110000045, 6110000052, 6110000053, 6110000054, 6110000055, 6110000056, 6110000059, 6110000060, 6110000066, 6110000072, 6110000080, 6110000082, 6110000083, 6110000084, 6110000093, 6110000094, 6110000100, 6110000104, 6110000107, 6110000109, 6110000115, 6110000116, 6110000121, 6110000125, 6110000127, 6110000128, 6110000129, 6110000130, 6110000136, 6110000137, 6110000141, 6110000171, 6130000005, 6130000007, 6170000078, 6170000079, 6170000080, 6170000085, 6170000086, 6170000116, 6170000117, 6170000118, 6170000119, 6170000175, 6180000001, 6180000002, 6180000003, 6240000001, 6240000005, 6240000006, 6240000010, 6240000011, 6240000012, 6240000013, 6240000014, 6240000015, 6270000008, 6270000010, 6270000011, 6270000012, 6270000013, 6310000001, 6310000002, 6340000004, 6340000019, 6380000003, 6400000001, 6400000002, 6400000003, 6400000006, 6400000007, 6410000001, 6410000002, 6410000003, 6410000004, 6410000005, 6410000006, 6440000002, 6440000003, 6440000004, 6500000013, 6500000014, 6500000015, 6530000001, 6530000002, 6550000001, 6550000002, 6550000003, 6550000004, 6550000005, 6550000006, 6550000007, 6550000008, 6550000009, 6570000001, 6570000002, 6590000001, 6590000002, 6620000022, 6620000045, 6620000046, 6620000047, 6620000048, 6620000049, 6620000050, 6620000051, 6620000052, 6620000053, 6620000054, 6620000055, 6620000056, 6620000057, 6620000058, 6620000060, 6620000061, 6620000062, 6620000063, 6620000064, 6620000076, 6620000077, 6620000078, 6620000079, 6620000080, 6620000081, 6620000082, 6620000085, 6620000091, 6620000092, 6620000093, 6620000095, 6620000096, 6620000097, 6620000098, 6620000101, 6620000109, 6620000110, 6620000142, 6620000143, 6620000144, 6620000145, 6620000149, 6620000150, 6620000151, 6620000152, 6620000159, 6630000001, 6630000002, 6630000003, 6650000011, 6650000012, 6650000015, 6670000004, 6670000011, 6690000001, 6690000002, 6690000003, 6690000004, 6690000005, 6690000006, 6700000001, 6700000002, 6700000003, 6710000001, 6710000002, 6710000004, 6750000002, 6750000003, 6750000004, 6750000013, 6750000014, 6750000015, 6760000001, 6780000001, 6780000002, 6780000003, 6780000004, 6780000006, 6780000007, 6780000009, 6780000010, 6780000011, 6840000004, 6840000005, 6840000006, 6840000007, 6860000001, 6890000011, 6900000001, 6900000002, 6900000003, 6900000008, 6900000009, 6900000010, 6900000011, 6900000012, 6900000013, 6900000014, 6900000019, 6900000020, 6900000021, 6900000024, 6900000025, 6900000026, 6900000027, 6900000028, 6900000029, 6900000030, 6900000031, 6900000032, 6900000033, 6900000034, 6900000035, 6900000037, 6900000038, 6900000040, 6900000041, 6900000044, 6900000045, 6900000047, 6900000049, 6900000052, 6900000054, 6900000073, 6900000080, 6900000081, 6900000084, 6900000090, 6900000093, 6900000094, 6900000095, 6900000098, 6900000102, 6900000103, 6900000104, 6900000105, 6900000108, 6900000109, 6900000110, 6900000113, 6900000114, 6900000127, 6900000128, 6900000129, 6900000131, 6900000134, 6900000140, 6900000142, 6900000143, 6900000144, 6900000145, 6900000149, 6900000150, 6900000152, 6900000153, 6900000154, 6900000155, 6900000156, 6900000157, 6900000158, 6900000161, 6900000162, 6900000164, 6900000165, 6900000166, 6900000168, 6900000170, 6900000171, 6900000172, 6900000173, 6900000177, 6900000178, 6900000188, 6900000190, 6900000191, 6900000199, 6900000200, 6900000202, 6900000203, 6900000205, 6900000210, 6900000214, 6900000215, 6900000216, 6900000220, 6900000221, 6900000222, 6900000223, 6900000225, 6900000238, 6900000240, 6900000241, 6900000242, 6900000244, 6900000250, 6900000251, 6900000263, 6900000268, 6900000269, 6900000275, 6900000277, 6900000279, 6900000298, 6900000305, 6910000001, 6910000002, 6910000003, 6910000004, 6910000005, 6910000016, 6910000017, 6910000018, 6910000019, 6910000020, 6910000021, 6910000023, 6910000027, 6910000029, 6910000036, 6910000038, 6910000041, 6910000044, 6910000047, 6910000051, 6910000054, 6910000056, 6910000058, 6910000060, 6910000062, 6910000063, 6910000065, 6910000067, 6910000069, 6910000071, 6910000073, 6910000075, 6910000076, 6910000078, 6910000080, 6910000082, 6910000084, 6910000086, 6910000116, 6910000117, 6910000118, 6910000119, 6910000120, 6910000121, 6910000122, 6910000127, 6910000128, 6910000129, 6910000130, 6910000131, 6910000132, 6910000133, 6910000134, 6910000135, 6910000136, 6910000137, 6910000140, 6910000141, 6910000142, 6910000144, 6910000145, 6910000146, 6910000148, 6910000150, 6910000151, 6910000152, 6910000153, 6910000154, 6910000155, 6910000158, 6910000159, 6910000160, 6910000161, 6910000167, 6910000169, 6910000193, 6910000255, 6910000256, 6910000257, 6910000260, 6910000261, 6910000262, 6910000263, 6910000264, 6910000265, 6910000267, 6910000268, 6910000271, 6910000272, 6910000274, 6910000276, 6910000278, 6910000279, 6910000281, 6910000282, 6910000283, 6910000285, 6910000287, 6910000288, 6910000289, 6910000290, 6910000291, 6910000292, 6910000293, 6910000294, 6910000296, 6910000297, 6910000298, 6910000299, 6910000300, 6910000301, 6910000302, 6910000303, 6910000306, 6910000307, 6910000308, 6910000309, 6910000317, 6910000319, 6910000322, 6910000324, 6910000326, 6910000327, 6910000328, 6910000343, 6910000380, 6910000438, 6910000448, 6910000449, 6910000457, 6910000476, 6910000491, 6910000492, 6910000493, 6910000494, 6910000495, 6910000497, 6910000498, 6910000500, 6910000501, 6910000502, 6910000507, 6910000508, 6910000509, 6910000510, 6910000511, 6910000512, 6910000515, 6910000516, 6910000517, 6910000518, 6910000519, 6910000520, 6910000521, 6910000525, 6910000526, 6910000527, 6910000528, 6910000529, 6910000530, 6910000531, 6910000534, 6910000535, 6910000538, 6910000539, 6910000540, 6910000541, 6910000542, 6910000543, 6910000544, 6920000001, 6920000002, 6920000006, 6920000007, 6950000001, 6960000001, 6960000002, 6970000001, 6970000002, 6970000003, 6970000004, 6970000005, 6970000006, 6980000004, 6980000005, 7340000772]

In [198]:
id_all_data[id_all_data.id.isin(unique_id2)]

,id,id_lat,id_lng,id_lat_mesh,id_lng_mesh,id_prefecture,pvrate,observed_max,observed_max2
3,14,35.09792,136.88363,35.10,136.88,愛知県,4.13,3556.2000,NaN
5,18,36.21099,137.97718,36.22,137.98,長野県,4.14,3620.2166,NaN
8,24,37.90155,139.09578,37.90,139.10,新潟県,4.40,3549.5460,NaN
9,28,33.95867,134.64792,33.96,134.64,徳島県,8.28,6715.8980,NaN
10,29,35.48403,135.48168,35.48,135.48,福井県,3.00,2157.2088,NaN
...,...,...,...,...,...,...,...,...,...
5044,6970000005,35.03034,135.97541,35.04,135.98,滋賀県,16.00,14144.3302,14713.0
5045,6970000006,35.03034,135.97541,35.04,135.98,滋賀県,16.00,12373.7614,13497.0
5046,6980000004,36.05804,136.22206,36.06,136.22,福井県,10.44,10143.6000,10161.8
5047,6980000005,36.08526,136.22025,36.08,136.22,福井県,20.00,19684.2000,19976.6


In [208]:
id_all_data.loc[id_all_data.id.isin(unique_id2),["id_lat","id_lng"]].head()

,id_lat,id_lng
3,35.09792,136.88363
5,36.21099,137.97718
8,37.90155,139.09578
9,33.95867,134.64792
10,35.48403,135.48168


In [213]:
import folium

# 地図の中心座標を設定 (ここでは最初の緯度・経度)
map_center = df.iloc[0][['id_lat', 'id_lng']].tolist()

# 地図を生成
m = folium.Map(location=map_center, zoom_start=6)

# 各地点にマーカーを追加
for _, row in id_all_data[id_all_data.id.isin(unique_id2)].iterrows():
    folium.Marker([row['id_lat'], row['id_lng']]).add_to(m)

m

# 上による欠損値がないpvの予測精度

In [4]:
unique_id = unique_id2
unique_id = [int(i) for i in unique_id]

folder_path = "H:/study/preprocessing_data/1_twoweeks_nv"
id_list = []
files = os.listdir(folder_path)
for file in files:
    id_list.append(int(file.split(".")[0]))

unique_id = list(set(unique_id)&set(id_list))

to_unique_id = [str(num).zfill(10) for num in unique_id]
df = utils.get_preprocessing_data(to_unique_id)

df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=["id","datetime"],inplace=True)

save_files2 = save_files[save_files.id.isin(unique_id)]
save_files2["datetime"] = pd.to_datetime(save_files2["datetime"])
df["datetime"] = pd.to_datetime(df["datetime"])
df3 = save_files2.merge(df[["id","datetime",'two_weeks_max',"nv","generation"]],on=["id","datetime"],how="left")

id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")
df3 = df3.merge(id_all_data[["id","observed_max2"]],on=["id"],how="left")

df3["pred*two_weeks_max"] = df3["pred"] * df3["two_weeks_max"]
df3["pred_interpolated*two_weeks_max"] = df3["pred_interpolated"] * df3["two_weeks_max"]

NameError: name 'save_files' is not defined

In [234]:
((df3["generation"]-df3["pred*two_weeks_max"]).abs()/df3["observed_max2"]).mean()*100

5.175156412853391

In [235]:
((df3["generation"]-df3["pred_interpolated*two_weeks_max"]).abs()/df3["observed_max2"]).mean()*100

5.186952384573374

In [9]:
((df3["generation"]-df3["pred_interpolated*two_weeks_max"]).abs()/df3["observed_max2"]).mean()*100

inf

In [239]:
df3.head()

,id,pred,pred_interpolated,datetime,two_weeks_max,nv,generation,observed_max2,pred*two_weeks_max,pred_interpolated*two_weeks_max
0,564,0.703025,0.70302,2013-08-15 07:00:00,2626.0,0.766,2011.5160,8703.2,1846.143282,1846.130520
1,568,0.823727,0.82373,2013-08-15 07:00:00,1343.0,0.809,1086.4870,8703.2,1106.265697,1106.269390
2,10000002,0.778326,0.77833,2013-08-15 07:00:00,616.6,0.898,553.7068,8703.2,479.915651,479.918278
3,10000005,0.736680,0.73668,2013-08-15 07:00:00,491.4,0.834,409.8276,8703.2,362.004655,362.004552
4,10000007,0.683547,0.68355,2013-08-15 07:00:00,1097.8,0.697,765.1666,8703.2,750.397589,750.401190


In [244]:
df3[["id","generation","datetime"]]

,id,generation,datetime
0,564,2011.5160,2013-08-15 07:00:00
1,568,1086.4870,2013-08-15 07:00:00
2,10000002,553.7068,2013-08-15 07:00:00
3,10000005,409.8276,2013-08-15 07:00:00
4,10000007,765.1666,2013-08-15 07:00:00
...,...,...,...
14208475,6970000004,512.6010,2014-07-31 18:00:00
14208476,6970000005,574.3140,2014-07-31 18:00:00
14208477,6970000006,574.1512,2014-07-31 18:00:00
14208478,6980000004,514.9144,2014-07-31 18:00:00


In [ ]:
df4 = df3.groupby('id').apply(utils.prev_30m_generation).reset_index(level=0, drop=True)

MemoryError: Unable to allocate 2.43 GiB for an array with shape (11, 29608480) and data type float64

,id,pred,pred_interpolated,datetime
0,6,0.532696,0.53270,2014-07-31 18:00:00
1,9,0.232816,0.23282,2014-07-31 18:00:00
2,12,0.665454,0.66545,2014-07-31 18:00:00
3,14,0.232200,0.23220,2014-07-31 18:00:00
4,17,0.409859,0.40986,2014-07-31 18:00:00
...,...,...,...,...
5092,6990000048,0.768087,0.76809,2014-07-31 18:00:00
5093,6990000049,0.768087,0.76809,2014-07-31 18:00:00
5094,6990000050,0.525790,0.52579,2014-07-31 18:00:00
5095,6990000051,0.525790,0.52579,2014-07-31 18:00:00
